# G2ConfigMgr

The g2configMgr API is used to add specific JSON configurations to the database,
so that they may be shared across remote systems.
Such configurations are added to the database,
and a configuration ID is created for each of them.
The system may then be configured with a specific configuratin ID
that points to one of those configurations.
That configuration ID will then be the shared global config.

## Prepare environment

In [ ]:
import com.senzing.g2.engine.G2ConfigMgr;
import com.senzing.g2.engine.G2ConfigMgrJNI;
import com.senzing.g2.engine.G2Config;
import com.senzing.g2.engine.G2ConfigJNI;
import com.senzing.g2.engine.Result;

### Helper class for Json Rendering

In [ ]:
%%loadFromPOM
<dependency>
    <groupId>org.glassfish</groupId>
    <artifactId>javax.json</artifactId>
    <version>1.1.4</version>
</dependency>

In [ ]:
import javax.json.*;
import static java.util.Collections.*;
import static javax.json.stream.JsonGenerator.PRETTY_PRINTING;

In [ ]:
public class JsonUtil {
    private static final JsonWriterFactory PRETTY_FACTORY
        = Json.createWriterFactory(singletonMap(PRETTY_PRINTING, true));
        
    private static final JsonWriterFactory UGLY_FACTORY
        = Json.createWriterFactory(emptyMap());
    
    public static String toJsonText(JsonValue val) {
        return toJsonText(val, true);
    }

    public static String toJsonText(JsonValue val, boolean prettyPrint) {
        JsonWriterFactory factory = (prettyPrint) ? PRETTY_FACTORY : UGLY_FACTORY;
        StringWriter sw = new StringWriter();
        JsonWriter writer = factory.createWriter(sw);
        writer.write(val);
        sw.flush();
        return sw.toString();
    }
    
    public static JsonObject parseJsonObject(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readObject();
  }

    public static JsonArray parseJsonArray(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readArray();
  }
    
    
}

In [ ]:
import java.util.UUID;
String str;
public static void RenderJSON(Object obj){
    str = obj.toString();
    JsonObject json = JsonUtil.parseJsonObject(str);
    String Config = JsonUtil.toJsonText(json, false);
    UUID id = UUID.randomUUID();
    String uuid = id.toString();
    String div = "<div id=\""+ uuid +"\" style=\"height:100%; width:100%; background-color: LightCyan\"></div>";
    display(div, "text/html");
    String jav = "require([\"https://rawgit.com/caldwell/renderjson/master/renderjson.js\"], function() {document.getElementById(\'"+ uuid +"\').appendChild(renderjson("+json+"))});";
    display(jav, "application/javascript");
}

### Initialize Senzing configuration

Using environment variables and default values, create `senzingConfigJson`.
This value is used when instantiating Senzing objects.

In [ ]:
// Get variables used in constructing Senzing Engine configuration.

String configPath = System.getenv("SENZING_ETC_DIR");
if (configPath == null) {
    configPath = "/etc/opt/senzing";
}

String supportPath = System.getenv("SENZING_DATA_VERSION_DIR");
if (supportPath == null) {
    supportPath = "/opt/senzing/data";
}

String g2Path = System.getenv("SENZING_G2_DIR");
if (g2Path == null) {
    g2Path = "/opt/senzing/g2";
}

String resourcePath = g2Path + "/resources";

String sqlConnection = System.getenv("SENZING_SQL_CONNECTION");
if (sqlConnection == null) {
    sqlConnection = "sqlite3://na:na@/var/opt/senzing/sqlite/G2C.db";
}

// Construct the JSON string used for Senzing Engine configuration.

String senzingConfigJson = "{"
   + "\"PIPELINE\": {"
   +     "\"CONFIGPATH\": \"" + configPath + "\","
   +     "\"SUPPORTPATH\": \"" + supportPath + "\","
   +     "\"RESOURCEPATH\": \"" + resourcePath + "\""
   + "},"
   + "\"SQL\": {"
   +     "\"CONNECTION\": \"" + sqlConnection + "\""
   + "}}";
   
RenderJSON(senzingConfigJson);

## G2Config

### Create configuration object

The G2Config API is used to create a new JSON configuration from the default template.
For more information, 
view the [senzing-G2Config-sdk-api-specification.ipynb](senzing-G2Config-sdk-api-specification.ipynb) notebook.

In [ ]:
String moduleName = "ExampleG2Config";
boolean verboseLogging = true;

G2Config g2config = new G2ConfigJNI();
int return_code = g2config.init(moduleName, senzingConfigJson, verboseLogging);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(return_code);

## G2ConfigMgr

The g2configMgr API is used to add specific JSON configurations to the database,
so that they may be shared across remote systems.
Such configurations are added to the database, and a configuration ID is created for each of them.
The system may then be configured with a specific configuratin ID that points to one of those configurations.
That configuration ID will then be the shared global config.

### G2ConfigMgr Initialization

To start using g2configMgr, you must first create and initialize an instance of the config manager.
This should be done once per process.

Create a new instance of the config manager and assign it to a variable.
Then, call the appropriate initialization method to initialize the config manager.

###### Parameters

- **moduleName:** (str) A short name given to this instance of the G2 engine (i.e. your G2Module object)
- **senzingConfigJson:** (JSON str) A JSON document containing system parameters.
- **verboseLogging:** (bool) A boolean which enables diagnostic logging - this will print a massive amount of information to stdout (default = False)
Calling these functions will return "0" upon success - useful for error handling.

In [ ]:
String moduleName = "ExampleG2Config";
boolean verboseLogging = true;

G2ConfigMgr g2configMgr = new G2ConfigMgrJNI();
int return_code = g2configMgr.init(moduleName, senzingConfigJson, verboseLogging);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(return_code);

### Managing configurations

Configuration JSON documents may be added to the datastore, so that they can be shared across remote systems.
The `create()` method makes a new in-memory configuration from the `g2config.json` file.

In [ ]:
Result<Long> config = new Result<Long>();
long configHandle;
int return_code = g2config.create(config);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else{
    configHandle = config.getValue();
    System.out.print(config.getValue());
}

Save the in-memory configuration to a string buffer.

In [ ]:
StringBuffer SenzingModelConfigJson = new StringBuffer();
int returnCode = g2config.save(configHandle, SenzingModelConfigJson);
if (returnCode != 0)
    System.out.print(g2config.getLastException());
else
    RenderJSON(SenzingModelConfigJson);

#### addConfig

Use `addConfig()` to add a configuration JSON document to the data repository.

###### Parameters

- **SenzingModelConfigJson:** (str) The configuration JSON document.
- **configComments:** (str) A free-form string of comments describing the configuration document.
- **configID:** (long) The returned configID for the new config document registered in the data store.
The function returns "0" upon success.

In [ ]:
String timeStamp = new java.text.SimpleDateFormat("yyyy.MM.dd.HH.mm.ss").format(new java.util.Date());
String configComments = "senzing-g2configMgr-reference.ipynb added at " + timeStamp;
Result<Long> configID = new Result<Long>();

int return_code = g2configMgr.addConfig(SenzingModelConfigJson.toString(), configComments, configID);
if (returnCode != 0)
    System.out.print(g2config.getLastException());
else
    System.out.print("CONFIG ID: " + configID.getValue());

#### getConfigList

Use `getConfigList()` to retrieve a list of the configuration JSON documents contained in the data repository.
###### Parameters

- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [ ]:
StringBuffer response = new StringBuffer();

int returnCode = g2configMgr.getConfigList(response);
if (returnCode != 0)
    System.out.print(g2config.getLastException());
else
    RenderJSON(response);

#### getConfig
Use `getConfig()` to retrieve a specific configuration JSON document from the data repository. 

###### Parameters

- **configID:** (long) The configID for the config document that you wish to retrieve.
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [ ]:
StringBuffer response = new StringBuffer();

int returnCode = g2configMgr.getConfig(configID.getValue(), response);
if (returnCode != 0)
    System.out.print(g2config.getLastException());
else
    RenderJSON(response);

### Default configurations

Multiple configuration JSON documents may be added to the datastore. Each of them may be referenced to start the engine and other API's with different configurations.

In order to specify that one of the configuration documents should be the global shared config, the config ID for that configuration document must be set as the default config ID.

#### setDefaultConfigID

Use `setDefaultConfigID()` to set the default configuration JSON document in the data repository. 

###### Parameters

- **configID:** (long) The configID for the config document that you wish to retrieve.

The function returns "0" upon success.

In [ ]:
int returnCode = g2configMgr.setDefaultConfigID(configID.getValue());
if (returnCode != 0)
    System.out.print(g2configMgr.getLastException());
else
    System.out.print(configID.getValue());

#### getDefaultConfigID

Use `getDefaultConfigID()` to retrieve a specific configuration JSON document from the data repository.

###### Parameters

- **configID:** (long) The configID for the config document that you wish to retrieve.


In [ ]:
Result<Long> configID = new Result<Long>();
int returnCode = g2configMgr.getDefaultConfigID(configID);
if(returnCode!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(configID.getValue());

## Cleanup

### getLastException

In [ ]:
System.out.println(g2config.getLastException());
System.out.print(g2configMgr.getLastException());

### clearLastException

In [ ]:
g2config.clearLastException();
g2configMgr.clearLastException();

### close

- **configHandle:** (long) the handle of the config object to get the config json data from

In [ ]:
g2config.close(configHandle);

### destroy

In [ ]:
int return_code = g2config.destroy();

if(return_code!=0)
    System.out.print(g2config.getLastException());

In [ ]:
int return_code = g2configMgr.destroy();

if(return_code!=0)
    System.out.print(g2configMgr.getLastException());